In [1]:
'''更新日志16July
这是个百分制的回归模型
加了模型评价：各种误差
'''

'更新日志16July\n加了模型评价：ROC\n'

In [2]:
'''更新日志10July
单通道测试。
打脸了，不用原始牙的数据也能完成评分。
'''

'更新日志10July\n单通道测试。\n打脸了，不用原始牙的数据也能完成评分。\n'

In [3]:
'''
单通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。
输入：预备体的体素模型
网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出
'''

'\n单通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。\n输入：预备体的体素模型\n网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出\n'

In [4]:
'''
# 安装环境
pip install trimesh
'''

'\n# 安装环境\npip install trimesh\n'

In [5]:
'''
# 安装环境
pip install torch torchvision torchaudio
'''

'\n# 安装环境\npip install torch torchvision torchaudio\n'

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import trimesh

In [7]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  



In [8]:
# 定义数据集
class TeethDataset(Dataset):
    def __init__(self, score_file_path, voxel_size=32):
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data = mesh_to_voxel(mesh, self.voxel_size)
        return torch.tensor(voxel_data[np.newaxis, :], dtype=torch.float32), torch.tensor([score], dtype=torch.float32)

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(score_file_path, voxel_size=32)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [9]:
# 定义模型
class Teeth3DCNN(nn.Module):
    def __init__(self):
        super(Teeth3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1)  # 单通道输入
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)  # 体素尺寸为32*32*32，经过两次pooling变为8*8*8
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建模型
model = Teeth3DCNN()
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

In [10]:
# 训练模型
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")


Epoch 1/25, Loss: 782.0253718024805
Epoch 2/25, Loss: 323.84128440053837
Epoch 3/25, Loss: 268.7339776691638
Epoch 4/25, Loss: 273.60097082037674
Epoch 5/25, Loss: 250.38869195235404
Epoch 6/25, Loss: 272.0196284494902
Epoch 7/25, Loss: 294.82995309327777
Epoch 8/25, Loss: 352.4310271614476
Epoch 9/25, Loss: 293.8909246294122
Epoch 10/25, Loss: 265.75113181064006
Epoch 11/25, Loss: 223.38386184290835
Epoch 12/25, Loss: 217.18040742372213
Epoch 13/25, Loss: 238.5659704710308
Epoch 14/25, Loss: 203.47678304973402
Epoch 15/25, Loss: 164.08232119208887
Epoch 16/25, Loss: 139.5494618290349
Epoch 17/25, Loss: 120.57860645494964
Epoch 18/25, Loss: 139.32636946126036
Epoch 19/25, Loss: 147.79630711204126
Epoch 20/25, Loss: 77.65706378535221
Epoch 21/25, Loss: 73.56100785575416
Epoch 22/25, Loss: 53.93296859138891
Epoch 23/25, Loss: 52.26486451688566
Epoch 24/25, Loss: 56.179766818096766
Epoch 25/25, Loss: 47.753531995572544


In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [12]:
# 模型评估
model.eval()
test_loss = 0.0
all_targets = []
all_outputs = []
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        all_targets.extend(targets.numpy().flatten())
        all_outputs.extend(outputs.numpy().flatten())
print(f"Test Loss: {test_loss / len(dataloader)}")

# 计算评价指标
mse = mean_squared_error(all_targets, all_outputs)
rmse = np.sqrt(mse)
mae = mean_absolute_error(all_targets, all_outputs)
r2 = r2_score(all_targets, all_outputs)
mape = np.mean(np.abs((np.array(all_targets) - np.array(all_outputs)) / np.array(all_targets))) * 100

n = len(all_targets)
k = sum(p.numel() for p in model.parameters())
aic = n * np.log(mse) + 2 * k
bic = n * np.log(mse) + k * np.log(n)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R^2: {r2}")
print(f"MAPE: {mape}%")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

Test Loss: 21.213950342253636
MSE: 21.421669006347656
RMSE: 4.628355026245117
MAE: 3.7404792308807373
R^2: 0.916086821970262
MAPE: 6.333620846271515%
AIC: 16891213.660458565
BIC: 42317163.72860732


In [13]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}')


Tooth 1: Score = 71.1989974975586
Tooth 2: Score = 67.66210174560547
Tooth 3: Score = 65.981689453125
Tooth 4: Score = 67.91707611083984
Tooth 5: Score = 69.86016082763672
Tooth 6: Score = 65.68864440917969
Tooth 7: Score = 69.61299896240234
Tooth 8: Score = 73.42537689208984
Tooth 9: Score = 79.44298553466797
Tooth 10: Score = 73.68801879882812
Tooth 11: Score = 48.06551742553711
Tooth 12: Score = 63.05569839477539
Tooth 13: Score = 63.011409759521484
Tooth 14: Score = 65.41475677490234
Tooth 15: Score = 55.31198501586914
Tooth 16: Score = 41.976200103759766
Tooth 17: Score = 58.05792999267578
Tooth 18: Score = 62.38078308105469
Tooth 19: Score = 65.21367645263672
Tooth 20: Score = 60.61567306518555
Tooth 21: Score = 67.18598175048828
Tooth 22: Score = 66.95287322998047
Tooth 23: Score = 70.90615844726562
Tooth 24: Score = 72.61844635009766
Tooth 25: Score = 60.11852264404297
Tooth 26: Score = 62.952232360839844
Tooth 27: Score = 69.94408416748047
Tooth 28: Score = 65.66524505615234
T

In [14]:
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )

Tooth 1: Score = 71.1989974975586 Err = -3.8010025024414062
Tooth 2: Score = 67.66210174560547 Err = -0.33789825439453125
Tooth 3: Score = 65.981689453125 Err = -4.018310546875
Tooth 4: Score = 67.91707611083984 Err = -1.0829238891601562
Tooth 5: Score = 69.86016082763672 Err = -6.139839172363281
Tooth 6: Score = 65.68864440917969 Err = 1.6886444091796875
Tooth 7: Score = 69.61299896240234 Err = 0.6129989624023438
Tooth 8: Score = 73.42537689208984 Err = 0.42537689208984375
Tooth 9: Score = 79.44298553466797 Err = 2.4429855346679688
Tooth 10: Score = 73.68801879882812 Err = -3.311981201171875
Tooth 11: Score = 48.06551742553711 Err = -5.934482574462891
Tooth 12: Score = 63.05569839477539 Err = 1.0556983947753906
Tooth 13: Score = 63.011409759521484 Err = 4.011409759521484
Tooth 14: Score = 65.41475677490234 Err = 2.4147567749023438
Tooth 15: Score = 55.31198501586914 Err = -1.6880149841308594
Tooth 16: Score = 41.976200103759766 Err = -6.023799896240234
Tooth 17: Score = 58.05792999267